In [1]:
from dotenv import load_dotenv
import os
import getpass

In [2]:
load_dotenv(override=True)

True

In [3]:

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

from langchain.chat_models import init_chat_model

model = init_chat_model("gemini-2.0-flash", model_provider="google_genai")

In [4]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage("Translate the following from English into Italian"),
    HumanMessage("hi!"),
]

model.invoke(messages)

AIMessage(content='Ciao!', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--22d9373b-58d0-4f0c-86da-5cd3d6f036a7-0', usage_metadata={'input_tokens': 9, 'output_tokens': 3, 'total_tokens': 12, 'input_token_details': {'cache_read': 0}})

In [ ]:
# all these are equivalent
print(model.invoke("Hello"))
print(model.invoke([{"role": "user", "content": "Hello"}]))
print(model.invoke([HumanMessage("Hello")]))

content='Hello there! How can I help you today?' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []} id='run--3b6aeaba-ffd3-4956-8ab2-490279175103-0' usage_metadata={'input_tokens': 1, 'output_tokens': 11, 'total_tokens': 12, 'input_token_details': {'cache_read': 0}}
content='Hello! How can I help you today?' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []} id='run--d96ad646-011b-476b-b9ee-b375bef1915a-0' usage_metadata={'input_tokens': 1, 'output_tokens': 10, 'total_tokens': 11, 'input_token_details': {'cache_read': 0}}
content='Hello! How can I help you today?' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'saf

In [9]:
messages_2 = [
    SystemMessage("You are a professional explainer who can explain anything in 3 sentences"),
    HumanMessage("explain what postgres is"),
]

### Streaming

In [11]:
for token in model.stream(messages_2):
    print(token.content, end="|")

Post|greSQL is a| powerful, open-source relational database management system (RDBMS). It uses SQL| to store and retrieve data, ensuring data integrity and reliability. Known for its extens|ibility and compliance with SQL standards, PostgreSQL is suitable for a wide range of applications, from small projects to enterprise-level systems.
|

### Prompt Templating

We’re currently passing a list of messages to the language model. The list is typically built from user input and application logic. This logic formats the raw input—often by adding system messages or filling in templates—into a message list.
LangChain’s prompt templates help automate this process. They take raw user input and output a prompt ready for the model.
Let’s create a prompt template that takes two inputs:
language: The target translation language
text: The text to translate

In [14]:
from langchain_core.prompts import ChatPromptTemplate
system_template = "Translate the following from English into {language}"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)


> **Note:** `ChatPromptTemplate` supports multiple message roles within a single template. In this example, we format the `language` parameter into the **system message**, and the `text` into a **user message**.

The input to this prompt template is a dictionary. You can test the template directly to see what it generates:

```python
prompt = prompt_template.invoke({
    "language": "Italian",
    "text": "hi!"
})

prompt
```


In [15]:
prompt = prompt_template.invoke({"language": "Italian", "text": "hi!"})

prompt

ChatPromptValue(messages=[SystemMessage(content='Translate the following from English into Italian', additional_kwargs={}, response_metadata={}), HumanMessage(content='hi!', additional_kwargs={}, response_metadata={})])

In [16]:
response = model.invoke(prompt)
print(response.content)

Ciao!
